# Multiscale MILP

This is a continuation of 'One Location, One Temporal Scale, One Operation, Linear Programming Example' [Example 1]. Refer Example 1 to learn the basics on how Energia models processes. 

In this example, we add another Process [Solar PV] and Storage [Li-ion Battery]. Technology choice is modeled using binaries. Moreover, the model is multiscale as the operational capacities are decision variables. 

## See Example 1

In [18]:
from energia import *

m = Model('example2')
m.q = Periods()
m.y = 4 * m.q
m.usd = Currency()

## Resources

### Conveniently declaring components

Use m.declare(\<Object Type\>, \<list of names\>) to declare a large number of objects in one step.

In [19]:
m.declare(Resource, ['power', 'wind', 'solar'])

### Set bounds on Resource flows

Unlike wind which has bound on the total consumption, we set a daily limit on solar energy. The same bound is repeated in each quarter. The following constraints are written.

$\mathbf{cons}_{solar, network, quarter_0} \leq 100$

$\mathbf{cons}_{solar, network, quarter_1} \leq 100$

$\mathbf{cons}_{solar, network, quarter_2} \leq 100$

$\mathbf{cons}_{solar, network, quarter_3} \leq 100$

In [20]:
m.solar.consume(m.q) <= 100
m.wind.consume <= 400
m.power.release.prep(180) >= [0.6, 0.7, 0.8, 0.3]

--- General Resource Balance for solar in (l, q): initializing constraint, adding consume(solar, l, q)
    Completed in 0.0003981590270996094 seconds
--- Binding consume in domain (solar, l, q)
    Completed in 7.843971252441406e-05 seconds
--- General Resource Balance for wind in (l, y): initializing constraint, adding consume(wind, l, y)
    Completed in 7.486343383789062e-05 seconds
--- Binding consume in domain (wind, l, y)
    Completed in 4.553794860839844e-05 seconds
--- General Resource Balance for power in (l, q): initializing constraint, adding release(power, l, q)
    Completed in 0.00010848045349121094 seconds
--- Binding release in domain (power, l, q)
    Completed in 0.0001277923583984375 seconds


## Operations 

### Capacity as a variable 

Here we want the optimization problem to determine the optimal capacity. Moreover, we set binaries to avoid the lower bound being adhered to if the process is not set up. 

If the bounds are meant to be compulsory limits, skip the .x 

In [21]:
m.wf = Process()
m.wf(m.power) == -1 * m.wind
m.wf.capacity.x <= 100
m.wf.capacity.x >= 10
m.capacity.show()

--- Binding capacity in domain (wf, l, y)
    Completed in 0.0001480579376220703 seconds
--- Binding capacity in domain (wf, l, y)
    Completed in 0.0001385211944580078 seconds


<IPython.core.display.Math object>

<IPython.core.display.Math object>

Unlike in Example 1, where the capacity was know, capacity is a variable here. 

Moreover, the expenditure associated with operating and capacitating are different

In [22]:
m.wf.operate.prep(norm=True) <= [0.9, 0.8, 0.5, 0.7]
m.wf.capacity[m.usd.spend] == 990637 + 3354
m.wf.operate[m.usd.spend] == 49
m.operate.show(True)

--- Binding operate in domain (wf, l, q)
    Completed in 0.00018477439880371094 seconds
--- Mapping operate: (wf, l, q) → (wf, l, y)
    Completed in 0.000s


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [23]:
m.pv = Process()
m.pv(m.power) == -1 * m.solar
m.pv.capacity.x <= 100
m.pv.capacity.x >= 10

m.pv.operate.prep(norm=True) <= [0.6, 0.8, 0.9, 0.7]
m.pv.capacity[m.usd.spend] == 567000 + 872046
m.pv.operate[m.usd.spend] == 90000

--- Binding capacity in domain (pv, l, y)
    Completed in 0.0001609325408935547 seconds
--- Binding capacity in domain (pv, l, y)
    Completed in 0.00010824203491210938 seconds
--- Binding operate in domain (pv, l, q)
    Completed in 0.00015354156494140625 seconds
--- Mapping operate: (pv, l, q) → (pv, l, y)
    Completed in 0.000s


### Storage Operation

energia now allows storing to require the use of other resources, example power for hydrogen cryogenic storage. 

Provide an equation similar to Process, in this case the basis is the stored resource 
If no other resource is provided, it is assumed to be the charging/discharging efficiency

Note that the following are created internally: 
1. auxilary resource  with name resource.stored 
2. charging and discharging processes as storage.charge and storage.discharge 

The parameters for each of these can be set individually, thus allowing for a wide range of modeling approaches 

In [24]:
m.lii = Storage()

m.lii(m.power) == 0.9
m.lii.capacity.x <= 100
m.lii.capacity.x >= 10

# m.lii.capacity >= 10
m.lii.capacity[m.usd.spend] == 1302182 + 41432

m.lii.inventory[m.usd.spend] == 2000
m.lii.charge.capacity <= 100
m.lii.discharge.capacity <= 100

--- Binding invcapacity in domain (power.lii, l, y)
    Completed in 0.0001308917999267578 seconds
--- Binding invcapacity in domain (power.lii, l, y)
    Completed in 9.059906005859375e-05 seconds
--- General Resource Balance for power.lii in (l, y): initializing constraint, adding inventory(power.lii, l, y)
--- Binding capacity in domain (lii.charge, l, y)
    Completed in 0.0001976490020751953 seconds
--- Binding capacity in domain (lii.discharge, l, y)
    Completed in 4.0531158447265625e-05 seconds


## Locating Operations

Operations can be located as 

operation.locate(\<list of locations\>)

or 

m.location.operations(\<list of operations\>)

They both do the same thing 

In [25]:
m.pv.locate(m.network)
m.network.locate(m.wf, m.lii)

--- General Resource Balance for power in (l, q): adding produce(power, l, q, operate, pv)
    Completed in 0.00012540817260742188 seconds
--- General Resource Balance for solar in (l, q): adding expend(solar, l, q, operate, pv)
    Completed in 8.487701416015625e-05 seconds
--- General Resource Balance for power in (l, q): adding produce(power, l, q, operate, wf)
    Completed in 8.654594421386719e-05 seconds
--- General Resource Balance for wind in (l, y): adding expend(wind, l, y, operate, wf)
    Completed in 5.53131103515625e-05 seconds
--- Assuming  power.lii inventory capacity is unbounded in (l, y)
--- Assuming inventory of power.lii is bound by inventory capacity in (l, y)
--- Assuming inventory of power.lii is bound by inventory capacity in (l, q)
--- Mapping inventory: (power.lii, l, q) → (power.lii, l, y)
    Completed in 0.000s
--- General Resource Balance for power.lii in (l, q): initializing constraint, adding inventory(power.lii, l, q)
    Completed in 0.000172138214111

## Inventory Balance

Inventory is passed on from one time period (t - 1) to the next (t) and hence features in the general resource balance for resource.stored 

In [26]:
m.inventory.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Optimize!

In [27]:
m.show()

# Mathematical Program for Program(example2)

<br><br>

## Index Sets

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [28]:
m.usd.spend.opt()

--- Mapping spend: (usd, l, y, capacity, wf) → (usd, l, y)
    Completed in 0.000s
--- Mapping spend: (usd, l, y, operate, wf) → (usd, l, y)
    Completed in 0.000s
--- Mapping spend: (usd, l, y, capacity, pv) → (usd, l, y)
    Completed in 0.000s
--- Mapping spend: (usd, l, y, operate, pv) → (usd, l, y)
    Completed in 0.000s
--- Mapping spend: (usd, l, y, invcapacity, power.lii) → (usd, l, y)
    Completed in 0.000s
--- Mapping spend: (usd, l, y, inventory, power.lii) → (usd, l, y)
    Completed in 0.000s
--- Generating Program(example2).mps
--- Creating gurobi model for Program(example2)
Read MPS format model from file Program(example2).mps
Reading time = 0.01 seconds
PROGRAM(EXAMPLE2): 85 rows, 78 columns, 198 nonzeros
--- Optimizing Program(example2) using gurobi
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, u

## Solution

### Inventory Profiles

The inventory maintained in each time period is:

In [29]:
m.inventory.sol()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

The amount charged into inventory is:

In [30]:
m.produce(m.power.lii, m.lii.charge.operate, m.q).sol()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

The amount discharged from inventory is:

In [31]:
m.produce(m.power, m.lii.discharge.operate, m.q).sol()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Integer Decisions 

All the operations are setup in this case

In [32]:
m.capacity.reporting.sol()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [33]:
m.capacity.sol()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>